In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Read COBOL and Python files
with open("./cobol.txt", "r") as f:
    cobol_code = f.read()

with open("./python.txt", "r") as f:
    python_code = f.read()

# Tokenize the code (replace with COBOL-specific tokenization if needed)
cobol_tokens = nltk.word_tokenize(cobol_code)
python_tokens = nltk.word_tokenize(python_code)

# Create a vocabulary (add logic to handle unique tokens from both codes)
vocabulary = set(cobol_tokens + python_tokens)

# Placeholder for vectorization (replace with actual embedding method)
token_vectors = {token: [1 for _ in range(10)] for token in vocabulary}

# Sequence preparation (replace with logic for fixed-length sequences)
cobol_seq1 = cobol_tokens[:10]  # First 10 tokens as a sequence
python_seq1 = python_tokens[:10]

print("COBOL Sequence:", cobol_seq1)
print("Python Sequence:", python_seq1)
print("Sample Token Vector:", token_vectors["MOVE"])


COBOL Sequence: ['identification', 'division', '.', 'program-id', '.', 'cobol', '.', 'procedure', 'division', '.']
Python Sequence: ['test_cases', '=', '[', 'dict', '(', 'p1Points=0', ',', 'p2Points=0', ',', 'score=']
Sample Token Vector: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [11]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Embedding

In [29]:
# ... Your preprocessing code (tokenization, etc.) ...

# Create a vocabulary set to store unique tokens
your_vocabulary = set()

# Add tokens from COBOL code
for token in cobol_tokens:
  your_vocabulary.add(token)

# Add tokens from Python code
for token in python_tokens:
  your_vocabulary.add(token)

# Define hyperparameters (assuming your_vocabulary is populated)
vocabulary_size = len(your_vocabulary)
embedding_dim = 256
lstm_units = 256


In [30]:
# Define maximum sequence length (consider the longest sequence in your data)
max_sequence_length = 20  # Replace with the actual maximum length

# Create the embedding layer
embedding_layer = Embedding(vocabulary_size, embedding_dim, input_length=max_sequence_length)

In [31]:
# Define the encoder as an LSTM layer
encoder = LSTM(lstm_units, return_sequences=True)

def encode(sequences):
  # Pass the sequences through the embedding layer and then the LSTM
  embedded_sequences = embedding_layer(sequences)
  encoded_sequences = encoder(embedded_sequences)
  return encoded_sequences

In [32]:
# Define the decoder LSTM with attention (replace with basic LSTM if not using attention)
decoder_lstm = LSTM(lstm_units, return_sequences=True)
attention_layer = tf.keras.layers.Attention()  # Replace with appropriate attention layer

def decode(encoded_sequence, target_sequences):
  # Define the decoder loop (replace with attention mechanism if used)
  for i in range(1, len(target_sequences)):
    # Use the previous predicted token and encoded sequence for prediction
    previous_token = target_sequences[i - 1]
    context_vector = encoder(encoded_sequence)  # Replace with attention output if used
    decoder_output = decoder_lstm(tf.expand_dims(previous_token, axis=0), context_vector)
    # Predict the next token based on the decoder output
    predicted_token = tf.argmax(decoder_output, axis=2)  # Replace with softmax for probability
  return predicted_token

In [83]:
# Define the model (replace with attention mechanism if used)
model = tf.keras.Sequential([
  embedding_layer,
  encoder,
  decoder_lstm,
  LSTM(1, return_sequences=True),
  # Dense(vocabulary_size, activation='softmax')  # Output layer for predicting next token
])

# Compile the model (adjust loss function and optimizer as needed)
model.compile(loss='categorical_crossentropy', optimizer='adam')

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-83-3341564207d6>, line 6)

In [84]:
import numpy as np

In [85]:
# Pad sequences with zeros (assuming sequences are NumPy arrays)
def pad_sequence(sequence, max_length):
  padded_sequence = np.zeros(max_length, dtype=str)
  padded_sequence[:len(sequence)] = sequence
  return padded_sequence

# Pad COBOL sequence
cobol_seq1_padded = pad_sequence(cobol_seq1, max_sequence_length)

# Pad Python sequence
python_seq1_padded = pad_sequence(python_seq1, max_sequence_length)

# Reshape sequences with batch dimension
cobol_seq1_reshaped = np.expand_dims(cobol_seq1_padded, axis=0)
python_seq1_reshaped = np.expand_dims(python_seq1_padded, axis=0)

# print(cobol_seq1_reshaped[0])

# Train the model with reshaped and padded sequences
# model.fit(cobol_seq1_reshaped, python_seq1_reshaped, epochs=10)

In [86]:
# Reshape COBOL sequence (add batch dimension and potentially pad)
cobol_seq1_reshaped = np.expand_dims(cobol_seq1[:max_sequence_length], axis=0)

# Reshape Python sequence (add batch dimension and potentially pad)
python_seq1_reshaped = np.expand_dims(python_seq1[:max_sequence_length], axis=0)

# Train the model with reshaped sequences
# model.fit(cobol_seq1_reshaped, python_seq1_reshaped, epochs=10)

In [87]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 256)           13387776  
                                                                 
 lstm_2 (LSTM)               (None, 20, 256)           525312    
                                                                 
 lstm_3 (LSTM)               (None, 20, 256)           525312    
                                                                 
 lstm_4 (LSTM)               (None, 20, 1)             1032      
                                                                 
Total params: 14439432 (55.08 MB)
Trainable params: 14439432 (55.08 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [88]:
cobol_seq = [cobol_tokens[i:i+max_sequence_length] for i in range(0, len(cobol_tokens), max_sequence_length)]
python_seq = [python_tokens[i:i+max_sequence_length] for i in range(0, len(python_tokens), max_sequence_length)]

In [89]:
cobol_seq = cobol_seq[:-1]
python_seq = python_seq[:-1]

In [90]:
python_seq_trimmed = python_seq[:len(cobol_seq)]

In [94]:
from gensim.models import Word2Vec

In [131]:
w2v_cobol = Word2Vec(cobol_tokens, vector_size=2048, workers=4)

KeyboardInterrupt: 

In [ ]:
w2v_python = Word2Vec(python_tokens, vector_size=2048, workers=4)

In [130]:
vec_seq_cobol = []
vec_seq_python = []

for cs, ps in zip(cobol_seq, python_seq):
  csv, psv = [], []
  for ct, pt in zip(cs, ps):
    if ct in w2v_cobol.wv and pt in w2v_python.wv:
      csv.append(w2v_cobol.wv[ct])
      psv.append(w2v_python.wv[pt])

  # print(len(csv), len(psv))
  if len(csv) == 10:
    vec_seq_cobol.append(csv)
    vec_seq_python.append(psv)


Streaming output truncated to the last 5000 lines.
0 0
1 1
0 0
1 1
0 0
1 1
1 1
1 1
2 2
2 2
2 2
0 0
2 2
2 2
0 0
0 0
1 1
1 1
1 1
1 1
2 2
4 4
2 2
5 5
3 3
2 2
0 0
0 0
1 1
2 2
1 1
0 0
1 1
0 0
1 1
0 0
3 3
0 0
0 0
0 0
0 0
0 0
1 1
0 0
1 1
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
1 1
1 1
1 1
0 0
0 0
0 0
2 2
1 1
2 2
2 2
2 2
2 2
1 1
1 1
0 0
1 1
0 0
0 0
0 0
1 1
0 0
1 1
0 0
0 0
1 1
1 1
0 0
0 0
0 0
2 2
0 0
1 1
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 2
2 2
1 1
0 0
0 0
1 1
0 0
0 0
0 0
0 0
3 3
0 0
0 0
0 0
1 1
0 0
2 2
0 0
3 3
0 0
0 0
1 1
1 1
3 3
0 0
0 0
0 0
12 12
11 11
10 10
7 7
2 2
0 0
0 0
0 0
0 0
1 1
0 0
0 0
1 1
1 1
1 1
0 0
1 1
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
2 2
0 0
0 0
1 1
1 1
1 1
1 1
4 4
2 2
1 1
3 3
1 1
0 0
1 1
0 0
0 0
1 1
0 0
0 0
1 1
0 0
0 0
1 1
0 0
1 1
2 2
1 1
4 4
1 1
1 1
3 3
0 0
2 2
0 0
1 1
2 2
0 0
4 4
0 0
2 2
1 1
1 1
2 2
0 0
2 2
0 0
0 0
1 1
1 1
0 0
0 0
1 1
1 1
0 0
1 1
0 0
0 0
1 1
0 0
2 2
0 0
2 2
0 0
2 2
0 0
0 0
0 0
0 0
2 2
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 1
0 0
3 3
0 0
0 0
0 0
0 0
1 1
0 0

KeyboardInterrupt: 

In [120]:
print(len(vec_seq_cobol), len(vec_seq_python))

0 0


In [103]:
input_dim = 128
latent_dim = 50
output_dim = 128

In [106]:
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Define the input sequence
encoder_inputs = Input(shape=(None, input_dim))

# Encoder LSTM
encoder_lstm = LSTM(units=latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder input sequence
decoder_inputs = Input(shape=(None, output_dim))

# Decoder LSTM
decoder_lstm = LSTM(units=latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# Dense layer for output
decoder_dense = Dense(output_dim, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
